# Geometry Optimization
In this notebook we will show how to optimize the geometry of a molecule using BigDFT.

Author note: this is related to some mini-protein calculations I want to do...

## System Setup
We will do an example of OH-. First we define the input positions.

In [15]:
from futile import YamlIO as Y

positions ="""
- O: [-0.931477999671, 0.004547999723, -0.117331000086]
- H: [-1.355753693973, -0.032108553467, 0.725181911626]
"""
posinp = {"positions": Y.load(stream=positions, doc_lists=False),
          "units":"angstroem"}

We might also like to check the length of the bond as we do the optimization.

In [16]:
def oh_length(posinp):
    from numpy import array
    from numpy.linalg import norm
    
    pos1 = posinp["positions"][0]
    pos1 = array([float(x) for x in pos1.items()[0][1]])
    pos2 = posinp["positions"][1]
    pos2 = array([float(x) for x in pos2.items()[0][1]])
    
    return norm(pos2 - pos1)

In [17]:
print(oh_length(posinp))

0.944024138499


## Calculation Conditions
Now we will setup the input file for geometry optimization.

In [18]:
from BigDFT import Inputfiles as I
inp = I.Inputfile()
inp.set_xc("PBE")
inp.set_hgrid(0.4)
inp.write_orbitals_on_disk()

We will give this run a background charge to balance out the negative ion.

In [19]:
inp["dft"]["qcharge"] = -1.0

Finally, the parameters for optimizing the geometry.

In [20]:
inp["geopt"] = {}
inp["geopt"]["method"] = "FIRE"

And a calculator to run the calculation.

In [21]:
from BigDFT import Calculators as C
code = C.SystemCalculator(omp=2, mpi_run="mpirun")

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun /Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/BuildFast/install/bin/bigdft


## Actual Run and Convergence
Now we will do the actual run and check the convergence.

In [22]:
result = code.run(name="opt", input=inp, posinp=posinp, run_dir="geom")

Creating the yaml input file "geom/opt.yaml"
Run directory geom
Executing command:  mpirun /Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/BuildFast/install/bin/bigdft -n opt -s Yes
('Found', 9, 'different runs')


In [25]:
try:
    result.geopt_plot()
except:
    print("No image support")

We can check the new bond length.

In [35]:
opt_length = oh_length(result.log["Atomic structure"])
print(opt_length)

0.978764297245


## Target System
Now we will try to investigate a more interesting system with these parameters. Our goal will be to compare a few different systems and see which has the lowest energy. 

Now let's read in the target system.

In [36]:
from numpy import zeros
positions = []
cmat = None
with open("afb.pdb") as ifile:
    for line in ifile:
        split = line.split()
        if split[0] == "HETATM":
            element = split[2][0]
            pos = split[5:8]
            positions.append({element:pos})
        elif split[0] == "CONECT":
            if cmat is None:
                cmat = zeros((len(positions),len(positions)))
            i = int(split[1]) - 1
            for j in [int(x)-1 for x in split[2:]]:
                cmat[i,j] = 1

We want to add a radical to each of the double bonds. So first we need to get a list of all atom pairs that are linked by a double bond.

In [37]:
bond_pairs = []
for i, atom_i in enumerate(positions):
    if atom_i.keys()[0] != "C":
        continue
    if sum(cmat[i,:]) == 4.0:
        continue
    for j, atom_j in enumerate(positions):
        if cmat[i,j] == 0:
            continue
        if atom_j.keys()[0] != "C":
            continue
        if sum(cmat[:,j]) == 4.0:
            continue
        if (j,i) not in bond_pairs:
            bond_pairs.append((i,j))

Finally, we need a reasonable method of putting a radical near the double bond. We can draw a line from the center of mass of the molecule to the bond, and project out a little ways.

In [38]:
from numpy import array
com = array([0.0, 0.0, 0.0])
for atom in positions:
    el, r = atom.items()[0]
    r = array([float(x) for x in r])
    com += r
com /= len(positions)

In [39]:
from numpy.linalg import norm
radical_positions = []
for pair in bond_pairs:
    pos1 = array([float(x) for x in positions[pair[0]].items()[0][1]])
    pos2 = array([float(x) for x in positions[pair[0]].items()[0][1]])
    bond_center = 0.5*(pos1 + pos2)
    draw_line = (bond_center - com)
    pos3 = draw_line + (draw_line/norm(draw_line)) * 1.0 + com
    
    radical_positions.append({"H":[str(x) for x in pos3]})

Now we merge these together into a list of positions we would like to compute.

In [41]:
new_systems = {}
new_systems["Original"] = positions
for pair, rpos in zip(bond_pairs,radical_positions):
    new_systems[str(pair[0])+"-"+str(pair[1])] = positions + [rpos]

And run

In [42]:
def compute_systems(geomdict, inp, code):
    energy_values = {}
    for label, geom in geomdict.items():
        if len(geom) == 35:
            inp["dft"]["qcharge"] = 0.0
        else:
            inp["dft"]["qcharge"] = -1.0
        posinp = {"positions":geom, "units":"angstroem"}
        result = code.run(name=label, input=inp, posinp=posinp, 
                          skip=False, run_dir="geom-target")
        energy_values[label] =  result.log["Energy (Hartree)"]
    return energy_values

In [17]:
energy_vals = compute_systems(new_systems, inp, code)